In [14]:
# ÌîÑÎ°úÏ†ùÌä∏ Î£®Ìä∏Î•º Python Í≤ΩÎ°úÏóê Ï∂îÍ∞Ä
import sys
import os
from pathlib import Path
import polars as pl

# ÌòÑÏû¨ ÎÖ∏Ìä∏Î∂ÅÏùò ÏúÑÏπòÏóêÏÑú ÌîÑÎ°úÏ†ùÌä∏ Î£®Ìä∏ Ï∞æÍ∏∞
current_dir = Path.cwd()
if 'examples' in str(current_dir):
    # examples Ìè¥ÎçîÏóêÏÑú Ïã§ÌñâÌïòÎäî Í≤ΩÏö∞
    project_root = current_dir.parent.parent
else:
    # ÌîÑÎ°úÏ†ùÌä∏ Î£®Ìä∏ÏóêÏÑú Ïã§ÌñâÌïòÎäî Í≤ΩÏö∞
    project_root = current_dir

# ÌîÑÎ°úÏ†ùÌä∏ Î£®Ìä∏Î•º Python Í≤ΩÎ°úÏóê Ï∂îÍ∞Ä
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# ÌïÑÏöîÌïú Î™®Îìà Í∞ÄÏ†∏Ïò§Í∏∞
from typing import List, Dict, Any, Optional
from datetime import datetime

from quantbt import (
    TradingStrategy,
    BacktestEngine,  
    
    # Í∏∞Î≥∏ Î™®ÎìàÎì§
    SimpleBroker, 
    BacktestConfig,
    UpbitDataProvider,
    
    # Ï£ºÎ¨∏ Í¥ÄÎ†®
    Order, OrderSide, OrderType,
)

In [23]:
class MultiSymbolSMAStrategy(TradingStrategy):
    """Î≥ÄÎèôÏÑ± Í∏∞Î∞ò Ïã¨Î≥º ÏÑ†ÌÉù SMA Ï†ÑÎûµ
    
    ÌïòÏù¥Î∏åÎ¶¨Îìú Î∞©Ïãù:
    - ÏßÄÌëú Í≥ÑÏÇ∞: Polars Î≤°ÌÑ∞Ïó∞ÏÇ∞ (SMA + Î≥ÄÎèôÏÑ±)
    - Ïã¨Î≥ºÍ∞Ñ ÎπÑÍµê: precompute_indicatorsÏóêÏÑú ÌÉÄÏûÑÏä§ÌÉ¨ÌîÑÎ≥Ñ Î≥ÄÎèôÏÑ± ÏàúÏúÑ Í≥ÑÏÇ∞
    - Ïã†Ìò∏ ÏÉùÏÑ±: Dict Native Î∞©Ïãù (Î≥ÄÎèôÏÑ± 1Îì± Ïã¨Î≥ºÎßå Í±∞Îûò)
    
    Îß§Ïàò: Í∞ÄÍ≤©Ïù¥ SMA15 ÏÉÅÌöå + Î≥ÄÎèôÏÑ± ÏàúÏúÑ 1Îì±
    Îß§ÎèÑ: Í∞ÄÍ≤©Ïù¥ SMA30 ÌïòÌöå + Ìè¨ÏßÄÏÖò Î≥¥Ïú†Ï§ë
    """
    
    def __init__(self, buy_sma: int = 15, sell_sma: int = 30, volatility_window: int = 14, symbols: List[str] = ["KRW-BTC", "KRW-ETH"]):
        super().__init__(
            name="VolatilityBasedMultiSymbolSMAStrategy",
            config={
                "buy_sma": buy_sma,
                "sell_sma": sell_sma,
                "volatility_window": volatility_window
            },
            position_size_pct=0.8,  # 80%Ïî© Ìè¨ÏßÄÏÖò
            max_positions=1
        )
        self.buy_sma = buy_sma
        self.sell_sma = sell_sma
        self.volatility_window = volatility_window
        self.symbols = symbols
        
    def calculate_volatility(self, prices: pl.Series, window: int = 14) -> pl.Series:
        """Î°§ÎßÅ ÌëúÏ§ÄÌé∏Ï∞® Í∏∞Î∞ò Î≥ÄÎèôÏÑ± Í≥ÑÏÇ∞"""
        returns = prices.pct_change()
        return returns.rolling_std(window_size=window)
        
    def _compute_indicators_for_symbol(self, symbol_data):
        """Ïã¨Î≥ºÎ≥Ñ Í∏∞Î≥∏ ÏßÄÌëú Í≥ÑÏÇ∞ (Polars Î≤°ÌÑ∞ Ïó∞ÏÇ∞)"""
        
        # ÏãúÍ∞ÑÏàú Ï†ïÎ†¨ ÌôïÏù∏
        data = symbol_data.sort("timestamp")
        
        # Îã®Ïàú Ïù¥ÎèôÌèâÍ∑† Í≥ÑÏÇ∞
        buy_sma = self.calculate_sma(data["close"], self.buy_sma)
        sell_sma = self.calculate_sma(data["close"], self.sell_sma)
        
        # Î≥ÄÎèôÏÑ± Í≥ÑÏÇ∞ (ÌëúÏ§ÄÌé∏Ï∞® Í∏∞Î∞ò)
        volatility = self.calculate_volatility(data["close"], self.volatility_window)
        
        # ÏßÄÌëú Ïª¨Îüº Ï∂îÍ∞Ä
        return data.with_columns([
            buy_sma.alias(f"sma_{self.buy_sma}"),
            sell_sma.alias(f"sma_{self.sell_sma}"),
            volatility.alias("volatility")
        ])
    
    # precompute_indicatorsÎäî BaseStrategyÏóêÏÑú ÌëúÏ§Ä 2Îã®Í≥Ñ Ï≤òÎ¶¨Î°ú ÏûêÎèô Ïã§ÌñâÎê®
    # 1Îã®Í≥Ñ: _compute_indicators_for_symbol (Ïã¨Î≥ºÎ≥Ñ ÏßÄÌëú)
    # 2Îã®Í≥Ñ: _compute_cross_symbol_indicators (Ïã¨Î≥ºÍ∞Ñ ÎπÑÍµê)
    
    def _compute_cross_symbol_indicators(self, data: pl.DataFrame) -> pl.DataFrame:
        """Ïã¨Î≥ºÍ∞Ñ ÎπÑÍµê ÏßÄÌëú Í≥ÑÏÇ∞ - ÏãúÍ∞Ñ ÎèôÍ∏∞Ìôî Î≥¥Ïû• (ÏôÑÏ†Ñ Î≤°ÌÑ∞ Ïó∞ÏÇ∞)"""
        
        # üöÄ ÏôÑÏ†Ñ Î≤°ÌÑ∞ Ïó∞ÏÇ∞ Î∞©Ïãù: window function ÌôúÏö©
        ranked_data = data.with_columns([
            # ÌÉÄÏûÑÏä§ÌÉ¨ÌîÑÎ≥ÑÎ°ú Î≥ÄÎèôÏÑ± ÏàúÏúÑ Í≥ÑÏÇ∞ (None/NaNÏùÑ infÎ°ú Ï≤òÎ¶¨)
            pl.col("volatility")
            .fill_null(float('inf'))
            .fill_nan(float('inf'))
            .rank("ordinal")
            .over("timestamp")  # ÌÉÄÏûÑÏä§ÌÉ¨ÌîÑÎ≥Ñ ÏúàÎèÑÏö∞ Ìï®Ïàò
            .alias("vol_rank"),
            
            # ÌÉÄÏûÑÏä§ÌÉ¨ÌîÑÎ≥ÑÎ°ú ÏµúÏÜå Î≥ÄÎèôÏÑ±Ïù∏ÏßÄ ÌåêÎã®
            (pl.col("volatility") == pl.col("volatility").min().over("timestamp"))
            .alias("is_lowest_volatility")
        ])
        
        return ranked_data
    
    def generate_signals_dict(self, current_data: Dict[str, Any]) -> List[Order]:
        """Î≥ÄÎèôÏÑ± ÏàúÏúÑ Í∏∞Î∞ò ÌïÑÌÑ∞ÎßÅÏù¥ Ï†ÅÏö©Îêú Ïã†Ìò∏ ÏÉùÏÑ±"""
        orders = []
        
        if not self.broker:
            return orders
        
        symbol = current_data['symbol']
        current_price = current_data['close']
        buy_sma = current_data.get(f'sma_{self.buy_sma}')
        sell_sma = current_data.get(f'sma_{self.sell_sma}')
        vol_rank = current_data.get('vol_rank', 999)
        
        # Î≥ÄÎèôÏÑ± ÏàúÏúÑ Í∏∞Î∞ò ÌïÑÌÑ∞ÎßÅ (1Îì±Îßå Í±∞Îûò)
        if vol_rank != 1:
            return orders  # Î≥ÄÎèôÏÑ± 1Îì±Ïù¥ ÏïÑÎãàÎ©¥ Í±∞Îûò Ï§ëÎã®
        
        # ÏßÄÌëúÍ∞Ä Í≥ÑÏÇ∞ÎêòÏßÄ ÏïäÏùÄ Í≤ΩÏö∞ Í±¥ÎÑàÎõ∞Í∏∞
        if buy_sma is None or sell_sma is None:
            return orders
        
        current_positions = self.get_current_positions()
        
        # Îß§Ïàò Ïã†Ìò∏: Í∞ÄÍ≤©Ïù¥ SMA15 ÏÉÅÌöå + Ìè¨ÏßÄÏÖò ÏóÜÏùå + Î≥ÄÎèôÏÑ± 1Îì±
        if current_price > buy_sma and symbol not in current_positions:
            portfolio_value = self.get_portfolio_value()
            quantity = self.calculate_position_size(symbol, current_price, portfolio_value) / len(self.symbols)
            
            if quantity > 0:
                order = Order(
                    symbol=symbol,
                    side=OrderSide.BUY,
                    quantity=quantity,
                    order_type=OrderType.MARKET
                )
                orders.append(order)
                # print(f"üéØ Î≥ÄÎèôÏÑ± 1Îì± Îß§Ïàò: {symbol} @ {current_price:,.0f}Ïõê (SMA{self.buy_sma}: {buy_sma:,.0f}, VolÏàúÏúÑ: {vol_rank})")
        
        # Îß§ÎèÑ Ïã†Ìò∏: Í∞ÄÍ≤©Ïù¥ SMA30 ÌïòÌöå + Ìè¨ÏßÄÏÖò ÏûàÏùå (Î≥ÄÎèôÏÑ± ÏàúÏúÑ Î¨¥Í¥Ä)
        elif current_price < sell_sma and symbol in current_positions and current_positions[symbol] > 0:
            order = Order(
                symbol=symbol,
                side=OrderSide.SELL,
                quantity=current_positions[symbol],
                order_type=OrderType.MARKET
            )
            orders.append(order)
            # print(f"üìâ Îß§ÎèÑ Ïã†Ìò∏: {symbol} @ {current_price:,.0f}Ïõê (SMA{self.sell_sma}: {sell_sma:,.0f})")
        
        return orders


In [24]:


# 1. ÏóÖÎπÑÌä∏ Îç∞Ïù¥ÌÑ∞ ÌîÑÎ°úÎ∞îÏù¥Îçî
print("üîÑ Îç∞Ïù¥ÌÑ∞ ÌîÑÎ°úÎ∞îÏù¥Îçî Ï¥àÍ∏∞Ìôî Ï§ë...")
upbit_provider = UpbitDataProvider()

# 2. Î∞±ÌÖåÏä§ÌåÖ ÏÑ§Ï†ï 
config = BacktestConfig(
    symbols=["KRW-BTC", 'KRW-ETH'],
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2024, 12, 31), 
    timeframe="1d",  # 1ÏãúÍ∞ÑÎ¥â (1Î∂ÑÎ¥âÎ≥¥Îã§ Îπ†Î¶Ñ)
    initial_cash=10_000_000,  # 1Ï≤úÎßåÏõê
    commission_rate=0.0,      # ÏàòÏàòÎ£å 0% (ÌÖåÏä§Ìä∏Ïö©) - Ïã§Ï†ú Î∞±ÌÖåÏä§ÌåÖÏóêÎäî Ï†ÅÏ†àÌïú Í∞í ÏÇ¨Ïö©
    slippage_rate=0.0,         # Ïä¨Î¶¨ÌîºÏßÄ 0% (ÌÖåÏä§Ìä∏Ïö©) - Ïã§Ï†ú Î∞±ÌÖåÏä§ÌåÖÏóêÎäî Ï†ÅÏ†àÌïú Í∞í ÏÇ¨Ïö©
    save_portfolio_history=True
)

# 3. SMA Ï†ÑÎûµ
print("‚ö° Ï†ÑÎûµ Ï¥àÍ∏∞Ìôî Ï§ë...")
strategy = MultiSymbolSMAStrategy(
    buy_sma=15,   # Îß§Ïàò: Í∞ÄÍ≤©Ïù¥ 15ÏãúÍ∞Ñ Ïù¥ÌèâÏÑ† ÏÉÅÌöå
    sell_sma=30,   # Îß§ÎèÑ: Í∞ÄÍ≤©Ïù¥ 30ÏãúÍ∞Ñ Ïù¥ÌèâÏÑ† ÌïòÌöå
    symbols=["KRW-BTC", "KRW-ETH"]
)

# 4. Î∏åÎ°úÏª§ ÏÑ§Ï†ï
broker = SimpleBroker(
    initial_cash=config.initial_cash,
    commission_rate=config.commission_rate,
    slippage_rate=config.slippage_rate
)

# 5. Dict Native Î∞±ÌÖåÏä§Ìä∏ ÏóîÏßÑ (Phase 7)
print("üöÄ Dict Native Î∞±ÌÖåÏä§Ìä∏ ÏóîÏßÑ Ï¥àÍ∏∞Ìôî Ï§ë...")
engine = BacktestEngine()  # Dict Native ÏóîÏßÑ ÏÇ¨Ïö©!
engine.set_strategy(strategy)
engine.set_data_provider(upbit_provider)
engine.set_broker(broker)

üîÑ Îç∞Ïù¥ÌÑ∞ ÌîÑÎ°úÎ∞îÏù¥Îçî Ï¥àÍ∏∞Ìôî Ï§ë...
‚ö° Ï†ÑÎûµ Ï¥àÍ∏∞Ìôî Ï§ë...
üöÄ Dict Native Î∞±ÌÖåÏä§Ìä∏ ÏóîÏßÑ Ï¥àÍ∏∞Ìôî Ï§ë...


In [25]:
# 7. Í≤∞Í≥º Ï∂úÎ†•
result = engine.run(config)
    
# Í≤∞Í≥º ÏöîÏïΩ Ï∂úÎ†•
result.print_summary()

Î∞±ÌÖåÏä§ÌåÖ ÏßÑÌñâ: 100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 730/730 [00:00<00:00]

                 BACKTEST RESULTS SUMMARY
Period          : 2024-01-01 ~ 2024-12-31
Initial Capital : $10,000,000
Final Equity    : $16,444,311
Total Return    : 64.44%
Annual Return   : 64.50%
Volatility      : 23.73%
Sharpe Ratio    : 2.72
Calmar Ratio    : 3.28
Sortino Ratio   : 3.56
Max Drawdown    : 19.67%
Total Trades    : 28
Win Rate        : 57.1%
Profit Factor   : 5.98
Execution Time  : 0.02s


In [13]:
# 1. Ìè¨Ìä∏Ìè¥Î¶¨Ïò§ ÏÑ±Í≥º Ï∞®Ìä∏
result.plot_portfolio_performance()

# 2. ÏàòÏùµÎ•† Î∂ÑÌè¨ 
result.plot_returns_distribution(period="daily")

# 3. ÏõîÎ≥Ñ ÏàòÏùµÎ•† ÌûàÌä∏Îßµ
result.plot_monthly_returns_heatmap()

# 4. ÏÑ±Í≥º ÎπÑÍµêÌëú
result.show_performance_comparison()